In [1]:
import numpy as np
import matplotlib.pyplot as plt
import re
from scipy.signal import iirnotch, lfilter
import numpy as np
import matplotlib.pyplot as plt


Fs = 100
Ts = 1/Fs
d_emg1 = np.genfromtxt("./DIN_EMG_1.txt", delimiter="\t",skip_header = 3)
d_emg2 = np.genfromtxt("./DIN_EMG_2.txt", delimiter="\t",skip_header = 3)
emg1 = np.genfromtxt("./EMG_1.txt", delimiter="\t",skip_header = 3)
emg2 = np.genfromtxt("./EMG_2.txt", delimiter="\t",skip_header = 3)


In [2]:
wp=94
ws=157

# Parámetros del filtro
f0 = 40.0  # Frecuencia que queremos eliminar
Q = 30.0  # Calidad del filtro
fs = 100.0  # Frecuencia de muestreo

# Diseño del filtro
b, a = iirnotch(f0, Q, fs)



In [3]:
bits = 10 # Bits de la salida 
volt_range = 3.28 #Rango de voltaje en milivoltios (-1.64; 1.64) mV
# Extraer la penultima columna (datos)
s1 = np.vstack((d_emg1[: ,5], d_emg1[: ,11], d_emg1[: ,17]))
s2 = np.vstack((d_emg2[: ,5], d_emg2[: ,11], d_emg2[: ,17]))
s3 = np.vstack((emg1[: ,5], emg1[: ,11], emg1[: ,17]))
s4 = np.vstack((emg2[: ,5], emg2[: ,11], emg2[: ,17]))
d_y1 = (s1 * volt_range/(2**bits - 1)) # convertir los bits a mV
d_y2 = (s2 * volt_range/(2**bits - 1)) # convertir los bits a mV
y1 = (s3 * volt_range/(2**bits - 1)) # convertir los bits a mV
y2 = (s4 * volt_range/(2**bits - 1)) # convertir los bits a mV
"""
y = y - np.mean(y) # Centrar la señal, ya que va de -1.64 a 1.64
y = lfilter(b, a, y)
t = np.arange(0, len(y)*Ts, Ts) # Crear vector de tiempo considerando periodo de muestreo
print(y)
plt.figure(figsize=(15, 15))
plt.plot(t, y, label="señal") # Señal completa
plt.title("Señal EMG capturada")
plt.grid(linestyle=":")
plt.xlabel("Tiempo (s)") 
plt.ylabel("Amplitud (mV)")
fig = plt.gcf()
fig.set_facecolor('white')
plt.savefig('complete_signal.png', transparent=False, )
plt.show()
"""
y = y1
y = y - np.mean(y) # Centrar la señal, ya que va de -1.64 a 1.64
y = lfilter(b, a, y)
t = np.arange(0, len(y)*Ts, Ts) # Crear vector de tiempo considerando periodo de muestreo
plt.figure(figsize=(15, 15))
plt.plot(t, y, label="señal") # Señal completa
plt.title("Señal EMG capturada")
plt.grid(linestyle=":")
plt.xlabel("Tiempo (s)") 
plt.ylabel("Amplitud (mV)")
fig = plt.gcf()
fig.set_facecolor('white')
plt.savefig('complete_signal.png', transparent=False, )
plt.show()

In [ ]:
# Crear una figura y una matriz de subgráficas (3x1)
fig, axs = plt.subplots(3, 1, figsize=(10, 15))

# Asignar tus vectores de señales a una lista para simplificar el código
signals_list = [d_y1, d_y2, y1, y2]
signal_titles = ['Braquiorradial', 'Extensor radial del carpo', 'Flexor radial del carpo'] # Modifica esto a tu conveniencia
titles = ['Dinamómetro: Sujeto 1', 'Dinamómetro: Sujeto 2', 'Sujeto 1', 'Sujeto 2']
selected_group = 3  # Selecciona el grupo que deseas graficar (0 a 3)

# Asume que ya tienes tus coeficientes de filtro b, a definidos en algún lugar de tu código

# Para el conjunto de señales seleccionado, filtra y centra cada una, luego dibuja las señales en su respectiva subgráfica
signals = signals_list[selected_group]
for j in range(3):
    # Centrar y filtrar
    signals[j, :] = signals[j, :] - np.mean(signals[j, :])  # Centrar la señal
    signals[j, :] = lfilter(b, a, signals[j, :])  # Filtrar la señal

    # Crear vector de tiempo considerando periodo de muestreo
    Ts = 1.0  # Asegúrate de definir tu valor correcto de Ts (periodo de muestreo)
    t = np.arange(0, len(signals[j, :]) * Ts, Ts)  # Crear vector de tiempo

    # Dibujar la señal
    axs[j].plot(t, signals[j, :], label=signal_titles[j])
    
    # Añade títulos a las subgráficas para identificarlas
    axs[j].set_title(f'{signal_titles[j]}')


    # Añade etiquetas a los ejes X e Y
    axs[j].set(xlabel='time (s)', ylabel='voltage (mV)')

# Ajustar los espacios entre las subgráficas para evitar que se solapen
plt.suptitle(titles[selected_group])
plt.tight_layout()
fig.set_facecolor('white')
plt.savefig('complete_signal.png', transparent=False, )
plt.show()

In [ ]:
def window_rms(a, window_size):
    a2 = np.power(a,2)
    window = np.ones(window_size)/float(window_size)
    return np.sqrt(np.convolve(a2, window, 'valid'))

window_size = 50  # Ajusta este valor al tamaño de tu ventana

# Crear una figura y 4 subgráficas (una por cada conjunto de señales)
fig, axs = plt.subplots(4, figsize=(15, 20))

# Asignar tus vectores de señales a una lista para simplificar el código
signals_list = [d_y1, d_y2, y1, y2]
titles = ['Dinamómetro: Sujeto 1', 'Dinamómetro: Sujeto 2', 'Sujeto 1', 'Sujeto 2']
legend_labels = ['Braquiorradial', 'Extensor cubital del carpo', 'Flexor radial del carpo']
for i, signals in enumerate(signals_list):
    for j in range(3):
        # Calcular RMS
        rms_signal = window_rms(signals[j, :], window_size)

        # Crear vector de tiempo considerando periodo de muestreo
        # Considera que el tamaño del RMS será menor que el de la señal original debido al tamaño de la ventana
        Ts = 1.0  # Asegúrate de definir tu valor correcto de Ts (periodo de muestreo)
        t = np.arange(0, len(rms_signal) * Ts, Ts)  # Crear vector de tiempo

        # Dibujar la señal RMS y agregar una etiqueta a la línea para la leyenda
        axs[i].plot(t, rms_signal, label=legend_labels[j])

    # Añade títulos a las subgráficas para identificarlas
    axs[i].set_title(titles[i])

    # Añade la leyenda a la subgráfica
    axs[i].legend()

# Añade etiquetas a los ejes X e Y
for ax in axs:
    ax.set(xlabel='time (s)', ylabel='RMS voltage (mV)')

# Ajustar los espacios entre las subgráficas para evitar que se solapen
fig.set_facecolor('white')

plt.tight_layout()
plt.savefig('complete_signal.png', transparent=False, )
plt.show()


In [ ]:
# Crear una figura
fig, ax = plt.subplots(figsize=(10, 6))

# Asignar tus vectores de señales a una lista para simplificar el código
signals_list = [y1, y2]
group_labels = ['Sujeto 1', 'Sujeto 2']
signal_labels = ['Braquiorradial', 'Extensor cubital del carpo', 'Flexor radial del carpo']
colors = ['red', 'green', 'blue']  # Define colores para las barras

# Calcula las amplitudes promedio para cada conjunto de señales
avg_amplitudes = []
for i, signals in enumerate(signals_list):
    subject_amps = []
    for j in range(3):
        amplitude = np.mean(np.abs(signals[j, :]))   # Calcula la amplitud
        subject_amps.append(amplitude)
    avg_amplitudes.append(subject_amps)

# Dibuja las barras superpuestas para cada grupo de amplitudes
bar_width = 0.2
x = np.arange(len(group_labels))  # Las localizaciones de las etiquetas en el eje x

for i in range(3):
    ax.bar(x + i*bar_width, [avg_amp[i] for avg_amp in avg_amplitudes], width=bar_width, color=colors[i], alpha=0.7, label=signal_labels[i])

# Ajusta las etiquetas del eje x para que aparezcan en el centro de los grupos de barras
ax.set_xticks(x + bar_width)
ax.set_xticklabels(group_labels)

# Añade la leyenda, títulos y etiquetas a los ejes
ax.legend()
ax.set_title('Amplitud promedio de las señales EMG')
ax.set_xlabel('Grupo')
ax.set_ylabel('Amplitud promedio (mV)')
fig.set_facecolor('white')
plt.savefig('complete_signal.png', transparent=False, )
# Muestra el gráfico
plt.show()


In [ ]:
N = 2**bits                                    

signal1 = y

signal_fft = np.fft.fft(signal1, N)           # fft magtinud
signal_fft = np.round(np.abs(signal_fft),3)[0:N//2] # nos quedamos con los componente de la derecha de la FFT
signal_aux = signal_fft/signal_fft.max()     # hallamos el maximo para pasar la magnitud a escala db

with np.errstate(divide='ignore'):
    signal_fft_db = 10*np.log10(signal_aux)  

F_list = np.linspace(0,Fs/2, N//2)
F = np.round(F_list[np.argmax(signal_fft_db)], 1)  
plt.plot(F_list, signal_fft_db)
plt.text(F,0, f"{F}Hz")
plt.grid(linestyle=":")
plt.ylabel("Magnitud (db)")
plt.xlabel("Frecuencias (Hz)")
plt.title("FFT en el decibelios")
plt.xlim([0,50])
plt.xticks(np.arange(0,50,10))
fig = plt.gcf()
fig.set_facecolor('white')
fig.set_size_inches(10, 7)
plt.savefig('fft.png', transparent=False, )
plt.show()